# **torchtext**

**torchtext**是一个适用于torch的nlp包，提供了许多预训练模型和其他的功能

In [1]:
import torch
import torchtext.vocab as vocab

In [2]:
# 查看预训练的模型
vocab.pretrained_aliases.keys()

dict_keys(['charngram.100d', 'fasttext.en.300d', 'fasttext.simple.300d', 'glove.42B.300d', 'glove.840B.300d', 'glove.twitter.27B.25d', 'glove.twitter.27B.50d', 'glove.twitter.27B.100d', 'glove.twitter.27B.200d', 'glove.6B.50d', 'glove.6B.100d', 'glove.6B.200d', 'glove.6B.300d'])

In [3]:
# 使用相应的预训练模型
cache_dir = 'C:/D/Files/data/dataset/glove/' # 第一次的存储位置
glove = vocab.pretrained_aliases['glove.6B.50d'](cache=cache_dir)

C:/D/Files/data/dataset/glove/glove.6B.zip: 862MB [06:36, 2.17MB/s]                                                    
100%|██████████████████████████████████████████████████████████████████████▉| 399733/400000 [00:30<00:00, 26567.10it/s]

In [4]:
print("一共包含%d个词。" % len(glove.stoi))

一共包含400000个词。


得到的结果具有三个重要的属性
- `stoi`:词到索引的字典
- `itos`:索引到词的列表
- `vectors`:词向量

In [5]:
# 求近似词
def knn(W, x, k):
    # 添加的1e-9是为了数值稳定性
    cos = torch.matmul(W, x.view((-1,))) / (
        (torch.sum(W * W, dim=1) + 1e-9).sqrt() * torch.sum(x * x).sqrt())
    _, topk = torch.topk(cos, k=k)
    topk = topk.cpu().numpy()
    return topk, [cos[i].item() for i in topk]

In [6]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.vectors,
                    embed.vectors[embed.stoi[query_token]], k+1)
    for i, c in zip(topk[1:], cos[1:]):  # 除去输入词
        print('cosine sim=%.3f: %s' % (c, (embed.itos[i])))

In [9]:
get_similar_tokens('china', 3, glove)

cosine sim=0.936: taiwan
cosine sim=0.896: chinese
cosine sim=0.892: beijing


In [10]:
# 求类比词
def get_analogy(token_a, token_b, token_c, embed):
    vecs = [embed.vectors[embed.stoi[t]] 
                for t in [token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.vectors, x, 1)
    return embed.itos[topk[0]]

In [16]:
get_analogy('beijing', 'china', 'tokyo', glove) # 'japan'

'japan'